In [ ]:
# data analysis libraries
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
from scipy.ndimage.filters import gaussian_filter1d

%matplotlib inline


In [ ]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# data import
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
subs = pd.read_csv('./data/submission.csv')

<br><br>
## Verkennende data analyse
<br><br>
Voor de verkennende analyse bekijken we de dataset waarmee we gaan werken. Om een goede analyse te kunnen doen is het van belang de data precies te snappen en voor de analyse een aantal aannames te doen. 

We beginnen met een globale analyse op de gehele dataset, van daaruit zoemen we in naar de structuur van de kolommen en zijn waardes.

Wanneer dit is gedaan kunnen we een aantal simpele plots neerzetten om verbanden te zoeken en aannames te doen.

<br><br>

In [ ]:
# met info kunnen we de datatype, het aantal null-waardes en de kolomnamen bekijken.

train.info()

print("\n")
print("-"*40)
print("\n")

test.info()

In [ ]:
# met describe bekijken we globaal de waarde in de kolommen

print(train.describe())

print("")
print("-"*40)
print("")

print(test.describe())

In [ ]:
# met head laten we de eerste 5 regels zien

print(train.head())
    
print("\n")
print("-"*40)
print("\n")

print(test.head())


In [ ]:
train["weight"] = train["TargetValue"]/ train["Population"] * 100

In [ ]:
train

In [ ]:
fig = px.pie(train, values='TargetValue', names='Target')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

In [ ]:
# @ todo

# hier moet nog een text komen over wat we zijn

In [ ]:
# @ todo

# hier moet nog een text komen over wat we in de plot willen laten zien

In [ ]:
# Aantal bevestigde gevallen COVID19 PLOT
x_con = train[train['Target'] == 'ConfirmedCases'].groupby('Country_Region')['TargetValue'].sum().sort_values().tail(15)
x_con = x_con.to_frame(name = 'TargetValue').reset_index()
fig = go.Figure(go.Bar(x=x_con["TargetValue"], y=x_con["Country_Region"], orientation='h'))
fig.update_layout(title='Aantal bevestigde gevallen COVID19')
fig.show()


# Top 15 meeste doden door COVID19 PLOT
x_fat = train[train['Target'] == 'Fatalities'].groupby('Country_Region')['TargetValue'].sum().sort_values().tail(15)
x_fat = x_fat.to_frame(name = 'TargetValue').reset_index()
fig = go.Figure(go.Bar(x=x_fat["TargetValue"], y=x_fat["Country_Region"], orientation='h'))
fig.update_layout(title='Top 15 meeste doden door COVID19')
fig.show()

# deze lists gaan we later gebruiken voor een verdere analyse
most_conf = list(x_con['Country_Region'])
most_deaths = list(x_fat['Country_Region'])


### Aantal bevestigde gevallen COVID19
Dit is een barplot met op de x as de kolom ConfirmedCases en op de y as Country_Region.
<br><br>

### Top 15 meeste doden door COVID19
Dit is een barplot met op de x as de kolom Fatalities en op de y as Country_Region. 

<br><br>
### Waarneming
De daadwerlijke cijfers liggen voor Amerika momenteel (20-05-2020) op 91.938 doden en 1.528.661 cases. Hieruit 
kunnen we concluderen dat er iets niet klopt aan de plot of dataset.

<br><br>

In [ ]:
train = train.replace(np.nan, '', regex=True)

filter1= train["Country_Region"] == 'US'
filter2= train["Province_State"] == ''

temp = train.where(filter1)
temp = temp.where(filter2)
temp = temp.dropna()

population = temp.groupby(['Province_State'])['Population'].mean().sum()

print("De populatie van US:")
print(population)
print()


filter1= train["Country_Region"] == 'US'

temp = train.where(filter1)

temp = temp.dropna()

population = temp.groupby(['Country_Region'])['Population'].mean().sum()

print("opgeteld alle unieke County_Region populaties:")
print(population)



<br><br>
De absolute populatie van Amerika is ongeveer 320 miljoen. 

Voor de locaties in deze dataset geldt dat er in de dataset met verschillende granulatiteit word gewerkt, zowel 
1. USA
2. USA, New York
3. USA , New York, New York 

komt voor in de dataset.

Dit zorgt ervoor dat als we USA selecteren we 3x dezelfde data selecteren.<br>

We kunnen duseen keuze maken welke granulariteit we gaan gebruiken. Voor de verkennende analyse kiezen we ervoor om Amerika als heel land te gebruiken. Wellicht kiezen we later voor een andere granulariteit, dit zal worden bepaald door de vraag die we willen beantwoorden.

<br><br>

In [ ]:
train = train.replace(np.nan, '', regex=True)

filter1= train["Target"] == 'Fatalities'
filter2= train["Province_State"] == ''

temp = train.where(filter1)
temp = temp.where(filter2)
temp = temp.dropna()
temp.head()

x_weight_f = temp.groupby(['Country_Region', 'Population'])['TargetValue'].sum().sort_values()
x_weight_f = x_weight_f.to_frame(name='TargetValue').reset_index()

x_weight_f["weight"] = x_weight_f["TargetValue"]/ x_weight_f["Population"] * 100

x_weight_f = x_weight_f.sort_values(by="weight", ascending=True)
x_weight_f1 = x_weight_f.tail(15)


fig = go.Figure(go.Bar(x=x_weight_f1["weight"], y=x_weight_f1["Country_Region"], orientation='h'))
fig.update_layout(title='Top 15 zwaarst getroffen landen op basis van populatie en aantal overledenen in percentage')
fig.show()


De oorspronkelijke weight die bij deze dataset hoort kunnen we niet gebruiken om de populatie van de landen te vergelijken met het aantal confirmedcases. Omdat er niet duidelijk is hoe de huidige weight berekend is gaan we hier niet mee werken, ook op het Kaggle forum valt niet duidelijk te vinden waarop deze weight gebaseerd is. Daarom hebben we zelf een weight gemaakt door de populatie van de landen te delen door de confirmed cases en fatalities. Hierboven zie je een top 15 van het percentage aantal overledenen ten opzichte van de populatie van het land. Hieronder geld dit voor de confirmed cases.

In [ ]:
train = train.replace(np.nan, '', regex=True)

filter1= train["Target"] == 'ConfirmedCases'
filter2= train["Province_State"] == ''

temp = train.where(filter1)
temp = temp.where(filter2)
temp = temp.dropna()
temp.head()

x_weight_c = temp.groupby(['Country_Region', 'Population'])['TargetValue'].sum().sort_values()
x_weight_c = x_weight_c.to_frame(name='TargetValue').reset_index()

x_weight_c["weight"] = x_weight_c["TargetValue"]/ x_weight_c["Population"] * 100

x_weight_c = x_weight_c.sort_values(by="weight", ascending=True)
x_weight_c1 = x_weight_c.tail(15)


fig = go.Figure(go.Bar(x=x_weight_c1["weight"], y=x_weight_c1["Country_Region"], orientation='h'))
fig.update_layout(title='Top 15 zwaarst getroffen landen op basis van populatie en aantal besmettingen in percentage')
fig.show()


Zoals je ziet in de bovenstaande 2 grafieken zie je dat Diamond Princess en Holy See etc. Diamond Princess is een cruiseschip waarbij een corona uitbraak heeft plaatsgevonden, Holy See is het vaticaanstad. Dit zijn allemaal plekken waarvan wij ze niet belangrijk genoeg vinden om in deze grafiek te tonen omdat de populatie van deze plekken dermate laag is dat dit het totaalpercentage besmette en overleden mensen beinvloed. Hieronder nogmaals de grafieken zonder deze plekken/landen.

In [ ]:
x_weight_f = x_weight_f.where((x_weight_f["Country_Region"] != "Diamond Princess") & (x_weight_f["Country_Region"] != "San Marino") & (x_weight_f["Country_Region"] != "Andorra")& (x_weight_f["Country_Region"] != "San Marino") & (x_weight_f["Country_Region"] != "Andorra") & (x_weight_f["Country_Region"] != "Andorra"))
x_weight_f2 = x_weight_f.tail(18)

fig = go.Figure(go.Bar(x=x_weight_f2["weight"], y=x_weight_f2["Country_Region"], orientation='h'))
fig.update_layout(title='Top 15 zwaarst getroffen landen op basis van populatie en aantal overledenen in percentage')
fig.show()

In [ ]:
x_weight_c = x_weight_c.where((x_weight_c["Country_Region"] != "Diamond Princess") & (x_weight_c["Country_Region"] != "San Marino") & (x_weight_c["Country_Region"] != "Holy See")& (x_weight_c["Country_Region"] != "Andorra") & (x_weight_c["Country_Region"] != "Monaco") & (x_weight_c["Country_Region"] != "Qatar") & (x_weight_c["Country_Region"] != "Maldives"))
x_weight_c2 = x_weight_c.tail(21)

fig = go.Figure(go.Bar(x=x_weight_c2["weight"], y=x_weight_c2["Country_Region"], orientation='h'))
fig.update_layout(title='Top 15 zwaarst getroffen landen op basis van populatie en aantal besmettingen in percentage')
fig.show()


Om te kijken of er nog meer van dit soort landen zijn die niet relevent zijn bij onze analyse hebben we hieronder een barchart gemaakt waarin  de 30 landen staan met de laagste populatie

In [ ]:
pop_df = train.where((train["Province_State"] == ''))
pop_df = pop_df[["Country_Region", "Population"]]
pop_df = pop_df.drop_duplicates()

pop_df = pop_df.sort_values(by="Population", ascending=False)
pop_df = pop_df.dropna()
pop_df = pop_df.tail(30)

fig = px.bar(pop_df, y='Population', x='Country_Region', text='Population')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Top 30 laagste populatie" )

fig.show()

### Externe factoren

    - Gemiddelde leeftijd
    - Aantal gesloten scholen
    - Obesitas
    - Aantal testen
    - Arm en rijk verdeling
    - Bevolkingsdichtheid
    - Aantal dode per vierkante kilometer per dag
    - Klimaat
    - Coordinaten van landen tenopzichte van elkaar



Verdere toelichting per externe factor.
- Toelichting
- Import
- Visualisatie
- Analyse


## Date restructure

Om landen in de tijd te kunnen analyseren moeten we zorgen dat de dagen de juiste schaal hebben.

Dit betekend dat we geen string maar een int gaan gebruiken. we gebruiken de eerste dag en de laatste dag die voorkomt in de dataset om te bepalen wat de range word, hier loop-en we doorheen om te kijken welke dagen voorkomen in de dataset. Met deze dagen bouwen we een dict op om de nieuwe index toe te passen 

> nergens word gehardcode zodat wanneer de dataset meer datums bevat de code blijft werken

> we gaan ervanuit dat er geen datums tussen de eerste dag en de laatste dag bestaan die in de dataset missen

In [ ]:
train_df = train
test_df = test


# - verwijderen om als int te kunnen casten
train_df["Date"] = train_df["Date"].replace({'-': ''}, regex=True)
test_df["Date"] = test_df["Date"].replace({'-': ''}, regex=True)

In [ ]:
# kolom casten naar int

train_df["Date"] = pd.to_numeric(train_df["Date"])
test_df["Date"] = pd.to_numeric(test_df["Date"])


In [ ]:
# wat is de range van datums

min_date = train_df['Date'].min()
max_date = test_df['Date'].max()
print(min_date)
print(max_date)

In [ ]:
dict_values = ""
custom_index = 0

for i in range(int(min_date), (int(max_date)+1)):
    
    # komt het getal in de dataset voor?
    if i in train_df['Date'].unique():
        
        # ja, voeg een entry toe aan de dict <   20200101: 1,   >
        dict_values += str(i) + ": " + str(custom_index) + ", "
        custom_index+=1
        
    elif i in test_df['Date'].unique():
        dict_values += str(i) + ": " + str(custom_index) + ", "
        custom_index+=1
        
exec("date_dict = {" + dict_values + "}")

In [ ]:
print(date_dict[20200501])

In [ ]:
train_df['Date'] = train_df['Date'].replace(date_dict)
test_df['Date'] = test_df['Date'].replace(date_dict)

In [ ]:
unique_locations = train_df.Country_Region.unique()
unique_targets = train_df.Target.unique()

print("There are "+str(len(unique_locations))+"\t unique locations")
print("There are "+str(len(unique_targets))+"\t unique Targets")


In [ ]:
import plotly.graph_objects as go
import random
r = lambda: random.randint(0,255)

In [ ]:
countries = \
    [
    "Netherlands",
    "Belgium",
    "Spain",
    "Germany",
    "Sweden",
    "United Kingdom",
    "Italy",
    "France"
    ]

for tar in unique_targets:
    for loc in countries: 
        filter1 = train_df["Target"]==tar
        filter2 = train_df["Province_State"]==''
        filter3 = train_df["Country_Region"]==loc
        filter4 = train_df["TargetValue"]!=0
        
        temp = train_df.where(filter1)
        temp = temp.where(filter2)
        temp = temp.where(filter3)
        temp = temp.dropna()
        
        
        temp2 = temp.where(filter4)
        print('day {} is the first day of a {} value in {}.'.format(int(temp2["Date"].min()), tar, loc))
        
        

        x = temp["Date"]
        y = temp["weight"]
        
        ysmoothed = gaussian_filter1d(y, sigma=6)

        temp['ysmooth'] = ysmoothed
  

        fig = px.line(temp, x=x, y=y, color='Country_Region', color_discrete_sequence=['rgb({},{},{})'.format(r(),r(),r())])
        fig.show()

In [ ]:


countries = \
    [
            "Belgium",
            "Spain",
            "United Kingdom",
            "Italy",
            "France"
            ]

for tar in unique_targets:
    
    
    for loc in countries: 
        filter1 = train_df["Target"]==tar
        filter2 = train_df["Province_State"]==''
        filter3 = train_df["Country_Region"]==loc
        filter4 = train_df["TargetValue"]!=0
        
        temp = train_df.where(filter1)
        temp = temp.where(filter2)
        temp = temp.where(filter3)
        temp = temp.dropna()
        
        
        temp2 = temp.where(filter4)
#         print('day {} is the first day of a {} value in {}.'.format(int(temp2["Date"].min()), tar, loc))
        
        

        x = temp["Date"]
        y = temp["weight"]
               
        
        ysmoothed = gaussian_filter1d(y, sigma=6)
        date_min = int(temp2["Date"].min())  
 
        real_peak_value = y.max()
        real_peak_index  = np.where(y == np.amax(y))[0][0]
        
        gaus_peak_value = ysmoothed.max()
        gaus_peak_index  = np.where(ysmoothed == np.amax(ysmoothed))[0][0]

        fig = go.Figure()
        
        fig.add_trace(go.Scatter(x=x, y=ysmoothed,
                    mode='lines',
                    name='lines', line=dict(color="black")))

        
        fig.add_trace(go.Scatter(x=[gaus_peak_index], y=[gaus_peak_value],
                    mode='markers', name='markers', marker=dict(size=12, symbol="x", opacity=1, color='black')))
        
        fig.add_trace(go.Scatter(x=x[date_min:], y=y[date_min:],
                    mode='markers', name='markers', marker=dict(size=4, opacity=0.5, color='red')))
        
      
        fig.add_trace(go.Scatter(x=[real_peak_index, date_min], y=[ysmoothed[real_peak_index],ysmoothed[date_min]],
                    mode='markers', name='markers', marker=dict(size=12, symbol="x", opacity=1, color='red')))
        
        

        fig.show()
        
        

        


In [ ]:
import random
countries = \
    [
    "Belgium",
    "Spain",
    "United Kingdom",
    "Italy",
    "France"
    ]


for tar in unique_targets:
    
    fig = go.Figure()
    
    for loc in countries: 
        filter1 = train_df["Target"]==tar
        filter2 = train_df["Province_State"]==''
        filter3 = train_df["Country_Region"]==loc
        filter4 = train_df["TargetValue"]!=0
        
        temp = train_df.where(filter1)
        temp = temp.where(filter2)
        temp = temp.where(filter3)
        temp = temp.dropna()
        
        
        temp2 = temp.where(filter4)
        print('day {} is the first day of a {} value in {}.'.format(int(temp2["Date"].min()), tar, loc))
        date_min = int(temp2["Date"].min())  
        
        df = temp  
        
        ysmoothed = gaussian_filter1d(df['weight'], sigma=6)
        df['ysmooth'] = ysmoothed
        
        
        
        
        
        fig.add_trace(go.Scatter(x=x, y=ysmoothed,
                    mode='lines',
                    name=loc, 
                    line=dict(
                        color='rgb({},{},{})'.format(r(),r(),r()))))

    fig.show()

In [ ]:
countries = \
    [
    "Belgium",
    "Spain",
    "United Kingdom",
    "Italy",
    "France"
    ]


for tar in unique_targets:
    apended_df = pd.DataFrame(columns=['Id', 'County', 'Province_State', 'Country_Region', 'Population',
       'Weight', 'Date', 'Target', 'TargetValue', 'ysmooth', 'cycle_index'])
    for loc in countries: 
        filter1 = train_df["Target"]==tar
        filter2 = train_df["Province_State"]==''
        filter3 = train_df["Country_Region"]==loc
        filter4 = train_df["TargetValue"]!=0
        
        temp = train_df.where(filter1)
        temp = temp.where(filter2)
        temp = temp.where(filter3)
        temp = temp.dropna()
        
        
        temp2 = temp.where(filter4)
        print('day {} is the first day of a {} value in {}.'.format(int(temp2["Date"].min()), tar, loc))
        date_min = int(temp2["Date"].min())  
        

        df = temp[date_min:]
        ysmoothed = gaussian_filter1d(df['weight'], sigma=6)
        df['ysmooth'] = ysmoothed
        df['cycle_index'] = [l for l in range(1, len(df)+1)]
        apended_df = pd.concat([apended_df,df])
    apended_df['cycle_index'].replace(np.nan, 0)    
    fig = px.line(apended_df, x='cycle_index', y='ysmooth', color='Country_Region')
    fig.show()


In [ ]:
train['Date'].max()